#Pip installs

In [2]:
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.2 MB/s eta 0:00:00


In [3]:
!pip install trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [4]:
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.9 MB/s eta 0:00:00


In [5]:
pip install accelerate

In [61]:
pip install wandb #This library provides access to the Weights and Biases library to capture various metrics, during the fine-tuning process.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [6]:
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

#Imports

In [7]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.colab import drive
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
from huggingface_hub import notebook_login
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
) # All the APIs for downloading and working with various pre-trained models that are in the huggingface model hub.

In [8]:

drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [9]:
file_names = [
    'drive/MyDrive/examify_eams_courses.csv',
    'drive/MyDrive/examify_eams_exams.csv',
    'drive/MyDrive/examify_eams_questions_answers_logs.csv',
    'drive/MyDrive/examify_eams_questions.csv',
    'drive/MyDrive/examify_eams_texts.csv'
]


courses = pd.read_csv(file_names[0])
exams = pd.read_csv(file_names[1], escapechar = '\\')
q_answers_logs = pd.read_csv(file_names[2], escapechar = '\\')
questions = pd.read_csv(file_names[3], escapechar = '\\')
texts = pd.read_csv(file_names[4])

In [10]:
pd.set_option('display.max_columns', None)

In [11]:
courses

,id,name,level,created_at,updated_at,for_sale,sort_order
0,1,Engels,vwo,2019-12-28 16:35:19,2022-08-27 11:41:11,1,3
1,3,Duits,vwo,2020-03-19 18:51:01,2022-08-27 11:41:11,1,3
2,4,Duits,havo,2020-03-19 20:01:54,2022-08-27 11:41:11,1,2
3,5,Spaans,vwo,2020-03-21 10:48:48,2022-08-27 11:41:11,0,3
4,6,Spaans,havo,2020-03-21 11:36:49,2022-08-27 11:41:11,0,2
5,7,Frans,vwo,2020-03-21 20:42:00,2022-08-27 11:41:11,1,3
6,8,Frans,havo,2020-03-22 11:55:25,2022-08-27 11:41:11,1,2
7,9,Nederlands,vwo,2020-09-21 15:58:02,2022-08-27 11:41:11,1,3
8,10,Nederlands,havo,2020-09-21 15:58:12,2022-08-27 11:41:11,1,2
9,11,Duits,mavo,2021-03-28 16:39:01,2022-08-27 11:41:11,0,1


In [12]:
exams.head()

,id,description,created_at,updated_at,course_id,year,tijdvak,norm,texts_content,for_free,visible,practice_mode_available,pdf_texts,only_for_question_types,p_score
0,1,NaN,2019-12-28 17:22:30,2023-09-04 18:13:50,1,2019,1,0.7,"[{""name"":""Less waste "",""printscreen"":"""",""max_w...",0,1,1,https://www.examenblad.nl/system/files/2019/ex...,0,69
1,3,NaN,2020-03-09 18:20:38,2023-09-04 18:25:10,1,2019,2,0.7,"[{""name"":""Joint efforts"",""printscreen"":"""",""max...",0,1,1,https://www.examenblad.nl/system/files/2019/ex...,0,0
2,5,NaN,2020-03-16 19:30:29,2023-02-22 20:10:55,1,2018,1,1.3,"[{""name"":""Factory workers in Asia""},{""name"":""L...",0,1,1,https://static.examenblad.nl/9336118/d/ex2018/...,0,62
3,6,NaN,2020-03-19 18:52:56,2023-09-04 18:36:25,3,2019,1,1.0,"[{""name"":""Othellotorte"",""printscreen"":"""",""max_...",0,1,1,https://www.examenblad.nl/system/files/2019/ex...,0,0
4,7,NaN,2020-03-19 20:06:02,2023-02-22 20:22:35,4,2018,1,0.6,"[{""name"":""Wetterschutz zum Sitzen"",""printscree...",0,1,1,https://static.examenblad.nl/9336118/d/ex2018/...,0,60


In [13]:
english_exams = exams[exams["course_id"] == 1]

In [14]:
eng_id_list = english_exams["id"].tolist()

In [15]:
q_answers_logs.head()

,id,user_id,question_id,answer_id,points,open_answer,practice_session_id,created_at,updated_at,is_final
0,562,42,18,NaN,0.0,7,50,2020-03-03 07:38:58,2020-11-15 10:53:01,1
1,614,42,2,NaN,1.0,haar dna,56,2020-03-11 17:51:23,2020-11-15 10:53:02,1
2,622,42,18,NaN,0.0,5,56,2020-03-11 17:51:44,2020-11-15 10:53:02,1
3,629,42,23,NaN,0.0,2,56,2020-03-11 17:51:52,2020-11-15 10:53:02,1
4,647,42,40,NaN,1.0,5,56,2020-03-11 17:52:19,2020-11-15 10:53:02,1


In [16]:
questions.head()

,text_id,question_nr,intro,outro,points,type_id,answers_content,explanation,created_at,updated_at,name,answer_type,id,open_answer_correction,outro_extra,extra_image,question_builder,open_answer_image,open_answer_image_max_width,count_only_correct,p_score
0,5,11,"""There was a mother. Who had a monster sleepin...",Geef antwoord in het Nederlands of in het Engels,1,6,[],<p>De vraag is hier wie dit ‘monster’ is. In a...,2019-12-29 12:21:04,2022-04-12 18:13:42,"Wie of wat is dit ""monster""?",open,2,<p>een antwoord met de volgende strekking:\r\n...,NaN,NaN,"[{""format"":""textarea"",""textarea"":""\""There was ...",NaN,NaN,0,53
1,5,16,"""Informed Consent"" (alinea 13)",Noteer het nummer van deze alinea op het antwo...,1,9,[],<p>De vraag is hier in welke alinea duidelijk ...,2019-12-29 17:57:54,2022-04-12 18:13:42,In welke alinea wordt duidelijk wat de titel v...,open,18,<p>(alinea) 9</p>\r\n\r\n<p>of</p>\r\n\r\n<p>(...,NaN,NaN,"[{""format"":""textarea"",""textarea"":""\""Informed C...",NaN,NaN,0,62
2,6,21,"“Properly taught or produced, the plays can co...",Noteer het nummer van deze alinea op het antwo...,1,9,[],<p>De vraag is in welke andere alinea deze opv...,2019-12-29 18:06:27,2022-04-12 18:13:42,In welke andere alinea wordt deze opvatting vo...,open,23,<p>(alinea) 10</p>,NaN,NaN,"[{""format"":""textarea"",""textarea"":""\u201cProper...",NaN,NaN,0,55
3,7,29,“One solution is obvious but improbable: for E...,Citeer deze voorwaarde of antwoord in het Nede...,1,6,[],<p>De vraag is hier aan welke voorwaarde moet ...,2019-12-29 18:18:20,2022-04-12 18:13:42,Aan welke voorwaarde genoemd in alinea 8-11 mo...,open,35,<p>(it will require) all their leagues to adop...,NaN,NaN,"[{""format"":""textarea"",""textarea"":""\u201cOne so...",NaN,NaN,0,76
4,8,33,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",1,9,[],<p>De vraag is of het in de tekst duidelijk wo...,2019-12-29 18:26:15,2022-04-12 18:13:42,Wordt in de tekst duidelijk waarom dit voor ha...,open,39,<p>Nee</p>,NaN,NaN,"[{""format"":""textarea"",""textarea"":""\u201copen s...",NaN,NaN,0,89


In [17]:
questions.columns

Index(['text_id', 'question_nr', 'intro', 'outro', 'points', 'type_id',
       'answers_content', 'explanation', 'created_at', 'updated_at', 'name',
       'answer_type', 'id', 'open_answer_correction', 'outro_extra',
       'extra_image', 'question_builder', 'open_answer_image',
       'open_answer_image_max_width', 'count_only_correct', 'p_score'],
      dtype='object')

In [18]:
q_answers_logs.head()

,id,user_id,question_id,answer_id,points,open_answer,practice_session_id,created_at,updated_at,is_final
0,562,42,18,NaN,0.0,7,50,2020-03-03 07:38:58,2020-11-15 10:53:01,1
1,614,42,2,NaN,1.0,haar dna,56,2020-03-11 17:51:23,2020-11-15 10:53:02,1
2,622,42,18,NaN,0.0,5,56,2020-03-11 17:51:44,2020-11-15 10:53:02,1
3,629,42,23,NaN,0.0,2,56,2020-03-11 17:51:52,2020-11-15 10:53:02,1
4,647,42,40,NaN,1.0,5,56,2020-03-11 17:52:19,2020-11-15 10:53:02,1


In [19]:
texts.head()

,id,exam_id,name,created_at,updated_at,myorder,printscreen,printscreen_width,printscreen_height,p_score
0,5,1,Hard work is in her DNA,2019-12-29 15:36:07,2022-04-12 18:13:42,5,NaN,0.0,0.0,65
1,6,1,Poor Shakespeare must be turning in his grave,2019-12-29 15:36:07,2022-04-12 18:13:42,6,NaN,0.0,0.0,61
2,8,1,Careless in Red,2019-12-29 15:36:07,2022-04-12 18:13:42,8,NaN,0.0,0.0,69
3,24,4,The jet set,2020-03-15 09:45:29,2022-04-06 19:53:17,1,NaN,0.0,0.0,90
4,25,4,Storm reports,2020-03-15 09:45:29,2022-04-06 19:53:17,2,NaN,0.0,0.0,65


In [20]:
merged_df = pd.merge(q_answers_logs, questions[['id', 'intro', 'outro', 'open_answer_correction', 'outro_extra', 'p_score']],
                    left_on='question_id', right_on='id', how='left')

# Drop the duplicate 'id' column
merged_df.drop('question_id', axis=1, inplace=True)

# Rename the columns from q_answer_logs
merged_df.rename(columns={'intro': 'question_intro', 'outro': 'question_outro', 'open_answer_correction': 'question_open_answer_correction',
                          'outro_extra': 'question_outro_extra', 'p_score': 'question_p_score'},
                 inplace=True)


In [21]:
merged_df.shape

(104424, 15)

In [22]:
merged_df["id_y"].value_counts()

1068    592
1070    552
1072    522
2       510
18      487
       ... 
3179      1
3178      1
3169      1
3166      1
3151      1
Name: id_y, Length: 1280, dtype: int64

In [23]:
eng_id_list

[1,
 3,
 5,
 20,
 39,
 44,
 45,
 46,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 112,
 137,
 157,
 162,
 186,
 201]

In [24]:
result_df = merged_df[merged_df['id_y'].isin(eng_id_list)]
result_df.shape

(1009, 15)

In [25]:
result_df

,id_x,user_id,answer_id,points,open_answer,practice_session_id,created_at,updated_at,is_final,id_y,question_intro,question_outro,question_open_answer_correction,question_outro_extra,question_p_score
18,969,65,NaN,0.0,alinea 3,84,2020-03-18 09:21:39,2020-11-15 10:53:05,1,157,“the old-fashioned feminist approach” (alinea 10),Noteer het nummer van deze alinea.,<p>(alinea) 1</p>\r\n\r\n<p>ook goed\r\n\t<br>...,NaN,43
54,1548,92,NaN,1.0,Nee,105,2020-03-19 14:16:57,2020-11-15 10:53:09,1,39,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",<p>Nee</p>,NaN,89
60,1604,96,NaN,1.0,Nee,108,2020-03-19 14:35:45,2020-11-15 10:53:10,1,39,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",<p>Nee</p>,NaN,89
126,2696,144,NaN,1.0,Nee,154,2020-03-30 13:54:07,2020-11-15 10:53:19,1,39,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",<p>Nee</p>,NaN,89
395,8948,269,NaN,1.0,nee,415,2020-11-09 10:30:44,2020-11-15 10:53:44,1,39,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",<p>Nee</p>,NaN,89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102434,702344,5225,NaN,0.0,alinea 12,102554,2024-03-01 12:25:28,2024-03-01 12:46:05,1,39,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",<p>Nee</p>,NaN,89
102440,702409,5226,NaN,1.0,nee,102555,2024-03-14 11:03:52,2024-03-14 11:07:25,1,39,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",<p>Nee</p>,NaN,89
102445,702473,5227,NaN,0.0,"Ja, alinea 1",102556,2024-04-14 10:31:57,2024-04-14 11:34:58,1,39,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",<p>Nee</p>,NaN,89
102452,702546,5228,NaN,1.0,nee,102557,2024-03-02 07:42:31,2024-03-02 07:46:25,1,39,“open space was crucial to her peace of mind” ...,"Zo nee, antwoord “Nee”. Zo ja, noteer het numm...",<p>Nee</p>,NaN,89


In [26]:
# Function to clean HTML tags, newline characters, and extra whitespaces
def clean_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    clean_text = soup.get_text(separator=' ')
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()
    return clean_text

# Apply the function to the column
result_df["question_open_answer_correction"] = result_df["question_open_answer_correction"].apply(clean_html_tags)


<ipython-input-26-6c748ce50c57>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df["question_open_answer_correction"] = result_df["question_open_answer_correction"].apply(clean_html_tags)


In [27]:
result_df["id_y"].value_counts()

39     407
157    337
112    265
Name: id_y, dtype: int64

# Transforming dataset to LLama2 format

INSTRUCTION:
Below is an answer given by a student. This is the model answer: ……. Grade the answer of the student based on the model answer.

INPUT:
This is the answer given by the student: ……….

OUTPUT:
This is how many points the student should get: ………..

In [28]:
result_df = result_df[["points", "question_open_answer_correction", "open_answer"]]

In [29]:
train_dataset = result_df.rename(columns={'points': 'grade', 'question_open_answer_correction': 'model_answer', 'open_answer': 'student_answer'})


In [30]:
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

#Imports

In [68]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from google.colab import drive
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
from huggingface_hub import notebook_login
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)

#Logging in on huggingface

In [32]:
notebook_login()

#Loading model and tokenizer

In [34]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf", use_fast = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [35]:
base_model = "meta-llama/Llama-2-7b-hf"
new_model = "Llama-2-7b-hf-Examify"

In [36]:
# 4-bit quantization via QLoRA allows efficient finetuning of huge LLM models on consumer hardware while retaining high performance.
# QLoRA quantizes a pre-trained language model to 4 bits and freezes the parameters. A small number of trainable Low-Rank Adapter layers are then added to the model.
# reduces memory and computational costs by representing weights and activations with lower-precision data types

bnb_config = BitsAndBytesConfig(load_in_4bit=True, # Theoretically optimal quantization data type for normally distributed data, reducing precision
                                bnb_4bit_quant_type = "nf4", # NF4 is flexible in representing a wide range of values, which can be helpful if your data varies a lot in magnitude.
                                bnb_4bit_compute_dtype = torch.float16, # Loaded in 4bit so less precision is acceptable, less float precision is also acceptable.
                                bnb_4bit_use_double_quant = True) # Mitigate the negative effects of aggressive quantization

# Use double quant if you have problems with memory, use NF4 for higher precision, and use a 16-bit dtype for faster finetuning

peft_config = LoraConfig(lora_alpha = 32,# Scaling factor for the weight matrices. alpha is a scaling factor that adjusts the magnitude of the combined result
                         lora_dropout = 0.05, # Regularizaiton technique.
                         r = 16, # Dimension of the low rank matrices learned during the finetuning process. As this value is increased, the number of parameters needed to be updated during the low-rank adaptation increases
                         bias = "none", # Will not be training the bias.
                         task_type = "CAUSAL_LM", # Task generation.
                         target_modules = ("up_proj", "down_proj", "k_proj", "q_proj", "v_proj", "o_proj")) # Which modules or layers of the model should be targeted for configuration. All linear layers
                         # Only attention blocks are q_proj and v_proj. But all linear layers result in better adaptation quality.


In [69]:
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config = bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [38]:
model = prepare_model_for_kbit_training(model)

#Test dataset

In [55]:
# Load dataset from the hub
dataset = load_dataset("databricks/databricks-dolly-15k", split="train")

print(f"dataset size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

dataset size: 15011
{'instruction': 'Which company built the F-22 Raptor?', 'context': "The Lockheed Martin F-22 Raptor is an American single-seat, twin-engine, all-weather stealth tactical fighter aircraft developed for the United States Air Force (USAF). As the result of the USAF's Advanced Tactical Fighter (ATF) program, the aircraft was designed as an air superiority fighter, but also has ground attack, electronic warfare, and signals intelligence capabilities. The prime contractor, Lockheed Martin, built most of the F-22's airframe and weapons systems and conducted final assembly, while Boeing provided the wings, aft fuselage, avionics integration, and training systems.  \n  \nThe aircraft first flew in 1997 and was variously designated F-22 and F/A-22 before it formally entered service in December 2005 as the F-22A. Although the USAF had originally planned to buy a total of 750 ATFs, the program was cut to 187 operational aircraft in 2009 due to high costs, a lack of air-to-air m

In [56]:
dataset

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 15011
})

In [57]:
def format_instruction(sample):
	return f"""### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
{sample['response']}

### Response:
{sample['instruction']}
"""

In [58]:
from random import randrange

print(format_instruction(dataset[randrange(len(dataset))]))


### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM.

### Input:
In 2021 Surrey County Council in England finally moved its headquarters back into its area of jurisdiction from its former location in the Royal Borough of Kingston upon Thames. The new headquarters, in the Surrey town of Reigate, are on a site previously occupied by Canon UK. Historically Kingston had been part of the county of Surrey until it was made part of Greater London in 1965. Postal addresses in Kingston still often include Surrey despite not being part of the county anymore.

### Response:
Is the headquarters of Surrey County Council in Surrey?



#Training arguments

In [63]:
training_arguments = TrainingArguments(
    output_dir = "./results", # Output directory where model predeictions and checkpoints will be stored.
    num_train_epochs = 4, # Number of training epochs (iteration of training data).
    per_device_train_batch_size = 10, # Batch size per GPU for training.
    gradient_accumulation_steps = 1, # Number of update steps to accumulate the gradients for.
    save_strategy = "epoch", # Save after every epoch.
    evaluation_strategy = "steps",
    eval_steps = 1000,
    logging_steps = 5, # Log on to the console on the progress every X steps.
    optim = "paged_adamw_8bit", # Optimizer
    learning_rate = 2e-4, # Learning rate
    weight_decay = 0.001, # Regularization technique
    lr_scheduler_type= "linear", # Adjust learning rate.
    max_grad_norm = 0.3, # Prevent exploding gradients
    warmup_ratio = 0.1, # How many training steps used for warmup.
    report_to = "wandb", # Report metrics to X.
)

#SFT trainer

In [64]:
from trl import SFTTrainer # Used for fine-tuning the transformer model, using reinforcement learning. Instruction dataset to perform this reinforcement learning and fine-tune the model.

max_seq_length = 2048 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset = dataset,
    peft_config = peft_config,
    dataset_text_field = "instruction",
    max_seq_length = 512,
    tokenizer = tokenizer,
    #formatting_func = format_instruction,
    args=training_arguments
)

trainer.train()
trainer.model.save_pretrained(new_model)

Map:   0%|          | 0/15011 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

#Results

In [48]:
def format_instruction(sample):
    return f"""### Instruction:
Below is an answer given by a student. This is the model answer: {sample['model_answer']}.
Grade the answer of the student based on the model answer.

### Input:
{sample['student answer']}

### Response:
{sample['grade']}
"""

In [ ]:
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer, max_length = 128)
result = pipe(format_instruction(data))
print(result[0]['generated_text'][len(instruction):])